In [5]:
%run ./3_DATA_PREPARATION.ipynb

In [23]:
# Regresion Lineal para toda tienda
results = {}
results_metrics = []

for store in df.Store.unique():

    store_series = df.loc[df['Store'] == store]
    store_series = store_series[['Date', 'Weekly_Sales']]
    store_series.dropna(inplace=True)
    store_series.columns = ['Date', 'Sales']
    store_series = store_series[["Sales", "Date"]].resample("W",on ="Date").sum().reset_index()
    #store_df["Date"]=pd.to_datetime(store_df["Date"])
    #store_series.set_index("Date", inplace=True)

    store_series['Time'] = np.arange(len(store_series.index)) # add time step column

    #https://medium.com/@Hong_Tang/timeseries-split-with-sklearn-tips-8162c83612b9

    # Time Series Split
    tscv = TimeSeriesSplit() # instantiate object from class. default parameter sets object to perform k=5 splits

    # Set date as index
    store_series.set_index('Date', inplace=True)
    store_series.sort_index(inplace=True)

    #prepare features dataframe (X) and target (y) as data series
    X = store_series.drop(labels=['Sales'], axis=1)
    y = store_series['Sales']

    # Split train test sets for both features and targets time-series class (tscv) class returns two arrays to mark train and test sets
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Summary of Train Test Split: number of records and date range for training and testing dataset.
    #print('The training dataset has', len(X_train), 'records, ranging from', pd.DataFrame(y_train).index.min(), 'to', pd.DataFrame(y_train).index.max())
    #print('The testing dataset has', len(X_test), 'records, ranging from', pd.DataFrame(y_test).index.min(), 'to', pd.DataFrame(y_test).index.max())

    #https://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html#sphx-glr-auto-examples-linear-model-plot-ols-py

    # Create linear regression object
    regr = LinearRegression()

    # Train the model using the training sets
    regr.fit(X_train.values, y_train.values)

    # Make predictions using the testing set
    y_pred = regr.predict(X_test.values)

    # model performance
    #results_metrics.append(store, evaluate(y_test.values, y_pred))
    #esults_metrics.append(store)
    results_metrics.append(evaluate(y_test.values, y_pred))

    # save results
    results[store] = [X_train, X_test, y_train, y_test, y_pred]


    ## TODO: formato con pandas

In [34]:
tmp = df.groupby(['Date','Store']).agg(Total_Sales = ('Weekly_Sales', 'sum') ).reset_index() # group by Store and aggregate the sum of the Weekly_Sales
top5 = tmp.sort_values(by=['Total_Sales'], ascending=False).head(5)['Store'].values # get top5 selling stores

"""
#bring top 5 selling stores data
store14 = tmp.query('Store==14')
store20 = tmp.query('Store==20')
store10 = tmp.query('Store==20')
store4 = tmp.query('Store==20')
store13 = tmp.query('Store==20')

top5 = array([14, 20, 10,  4, 13], dtype=int64)

"""

top5_results = []
top5_results_metrics = []

for store in top5:
    top5_results.append(results[store])

for store in top5:
    top5_results_metrics.append(results_metrics[store])

In [35]:
len(top5_results), top5_results[0]

(5,
 [            Time
  Date            
  2010-02-07     0
  2010-02-14     1
  2010-02-21     2
  2010-02-28     3
  2010-03-07     4
  ...          ...
  2012-04-22   115
  2012-04-29   116
  2012-05-06   117
  2012-05-13   118
  2012-05-20   119
  
  [120 rows x 1 columns],
              Time
  Date            
  2012-05-27   120
  2012-06-03   121
  2012-06-10   122
  2012-06-17   123
  2012-06-24   124
  2012-07-01   125
  2012-07-08   126
  2012-07-15   127
  2012-07-22   128
  2012-07-29   129
  2012-08-05   130
  2012-08-12   131
  2012-08-19   132
  2012-08-26   133
  2012-09-02   134
  2012-09-09   135
  2012-09-16   136
  2012-09-23   137
  2012-09-30   138
  2012-10-07   139
  2012-10-14   140
  2012-10-21   141
  2012-10-28   142,
  Date
  2010-02-07    2623469.95
  2010-02-14    1704218.84
  2010-02-21    2204556.70
  2010-02-28    2095591.63
  2010-03-07    2237603.75
                   ...    
  2012-04-22    1875686.44
  2012-04-29    1784029.95
  2012-05-06    19493

In [40]:
#for i in range(0, len(top5_results_metrics)): print(top5_results_metrics[i], top5_results[i])

{'MAE': 34262.95679967559, 'MSE': 1921978020.8566308, 'RMSE': 43840.36976186025, 'R2': 0.09781765260698139} [            Time
Date            
2010-02-07     0
2010-02-14     1
2010-02-21     2
2010-02-28     3
2010-03-07     4
...          ...
2012-04-22   115
2012-04-29   116
2012-05-06   117
2012-05-13   118
2012-05-20   119

[120 rows x 1 columns],             Time
Date            
2012-05-27   120
2012-06-03   121
2012-06-10   122
2012-06-17   123
2012-06-24   124
2012-07-01   125
2012-07-08   126
2012-07-15   127
2012-07-22   128
2012-07-29   129
2012-08-05   130
2012-08-12   131
2012-08-19   132
2012-08-26   133
2012-09-02   134
2012-09-09   135
2012-09-16   136
2012-09-23   137
2012-09-30   138
2012-10-07   139
2012-10-14   140
2012-10-21   141
2012-10-28   142, Date
2010-02-07    2623469.95
2010-02-14    1704218.84
2010-02-21    2204556.70
2010-02-28    2095591.63
2010-03-07    2237603.75
                 ...    
2012-04-22    1875686.44
2012-04-29    1784029.95
2012-05-06    